## 使用 Scikit-learn 中的 GradientBoostingRegressor 预测加利福尼亚房价

In [8]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
import time

### 1. 数据集导入

In [9]:
# 加载数据集
california = fetch_california_housing(as_frame=True)
X, y = california.data, california.target

# 显示前几行
print(X.head())
print(y.head())

   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  
0    -122.23  
1    -122.22  
2    -122.24  
3    -122.25  
4    -122.25  
0    4.526
1    3.585
2    3.521
3    3.413
4    3.422
Name: MedHouseVal, dtype: float64


### 2. 数据预处理

In [10]:
# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 特征缩放
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### 3. 使用 GradientBoostingRegressor 构建模型

In [11]:
# 初始化模型
gbr = GradientBoostingRegressor(random_state=42)

# 拟合模型
gbr.fit(X_train_scaled, y_train)

GradientBoostingRegressor(random_state=42)

### 4. 使用交叉验证评估模型

In [12]:
# 执行交叉验证
cv_scores = cross_val_score(gbr, X_train_scaled, y_train, cv=5, scoring='r2')

# 显示得分
print(f"交叉验证 R² 得分: {cv_scores}")
print(f"交叉验证 R² 平均得分: {cv_scores.mean():.4f}")

交叉验证 R² 得分: [0.78648904 0.78753035 0.79210152 0.78371007 0.78312941]
交叉验证 R² 平均得分: 0.7866


### 5. 超参数调优

我们将探索以下超参数的不同取值范围：

- **n_estimators**：提升阶段的数量。
- **learning_rate**：缩小每棵树的贡献。
- **max_depth**：单个回归估计器的最大深度。
- **min_samples_split**：分割内部节点所需的最小样本数。
- **subsample**：拟合单个基学习器时使用的样本比例。

#### 5.1 GridSearchCV初步调优

In [13]:
# 定义超参数网格
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'subsample': [0.8, 1.0]
}


# 初始化 GridSearchCV
grid_search = GridSearchCV(
    estimator=GradientBoostingRegressor(random_state=42),
    param_grid=param_grid,
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)

# 拟合 GridSearchCV
start_time = time.time()
grid_search.fit(X_train_scaled, y_train)
end_time = time.time()

print(f"网格搜索时间: {end_time - start_time:.2f}秒")
print(f"最佳参数: {grid_search.best_params_}")
print(f"最佳交叉验证 R² 得分: {grid_search.best_score_:.4f}")

Fitting 3 folds for each of 162 candidates, totalling 486 fits
网格搜索时间: 284.41秒
最佳参数: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 300, 'subsample': 0.8}
最佳交叉验证 R² 得分: 0.8357


#### 5.2 精细调优超参数空间

In [14]:
# 根据初步结果精细调整超参数网格
refined_param_grid = {
    'n_estimators': [180, 200, 220],
    'learning_rate': [0.04, 0.05, 0.06],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5],
    'subsample': [0.7, 0.8]
}

# 初始化精细调优的 GridSearchCV
refined_grid_search = GridSearchCV(
    estimator=GradientBoostingRegressor(random_state=42),
    param_grid=refined_param_grid,
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)

# 拟合精细调优的 GridSearchCV
start_time = time.time()
refined_grid_search.fit(X_train_scaled, y_train)
end_time = time.time()

print(f"精细网格搜索时间: {end_time - start_time:.2f}秒")
print(f"最佳参数: {refined_grid_search.best_params_}")
print(f"最佳交叉验证 R² 得分: {refined_grid_search.best_score_:.4f}")

Fitting 3 folds for each of 108 candidates, totalling 324 fits
精细网格搜索时间: 163.84秒
最佳参数: {'learning_rate': 0.06, 'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 220, 'subsample': 0.8}
最佳交叉验证 R² 得分: 0.8264


#### 5.3 提前停止

In [15]:
# 进一步划分训练数据集用于提前停止
X_train_es, X_val_es, y_train_es, y_val_es = train_test_split(
    X_train_scaled, y_train, test_size=0.1, random_state=42
)

# 初始化带提前停止的模型
gbr_es = GradientBoostingRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=4,
    min_samples_split=2,
    subsample=0.8,
    random_state=42,
    validation_fraction=0.1,
    n_iter_no_change=10
)

# 拟合模型
start_time = time.time()
gbr_es.fit(X_train_es, y_train_es)
end_time = time.time()

print(f"提前停止训练时间: {end_time - start_time:.2f}秒")
print(f"树的数量: {gbr_es.n_estimators_}")

提前停止训练时间: 14.69秒
树的数量: 528


### 6. 与其他模型的比较分析

#### 6.1. 随机森林回归器

In [16]:
# 初始化模型
rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=10,
    min_samples_split=2,
    random_state=42,
    n_jobs=-1
)

# 拟合模型
start_time = time.time()
rf.fit(X_train_scaled, y_train)
end_time = time.time()
rf_training_time = end_time - start_time

# 交叉验证
rf_cv_scores = cross_val_score(rf, X_train_scaled, y_train, cv=5, scoring='r2')

# 测试集 R² 得分
rf_test_score = rf.score(X_test_scaled, y_test)

print(f"随机森林训练时间: {rf_training_time:.2f}秒")
print(f"随机森林交叉验证 R² 得分: {rf_cv_scores}")
print(f"随机森林平均交叉验证 R² 得分: {rf_cv_scores.mean():.4f}")
print(f"随机森林测试集 R² 得分: {rf_test_score:.4f}")

随机森林训练时间: 1.57秒
随机森林交叉验证 R² 得分: [0.78067341 0.77591579 0.78752593 0.7846383  0.78224152]
随机森林平均交叉验证 R² 得分: 0.7822
随机森林测试集 R² 得分: 0.7749


#### 6.2. AdaBoost 回归器

In [17]:
# 初始化模型
ada = AdaBoostRegressor(
    n_estimators=200,
    learning_rate=0.1,
    random_state=42
)

# 拟合模型
start_time = time.time()
ada.fit(X_train_scaled, y_train)
end_time = time.time()
ada_training_time = end_time - start_time

# 交叉验证
ada_cv_scores = cross_val_score(ada, X_train_scaled, y_train, cv=5, scoring='r2')

# 测试集 R² 得分
ada_test_score = ada.score(X_test_scaled, y_test)

#### 6.3. 最终的 Gradient Boosting 回归器

In [18]:
# 使用 GridSearchCV 中的最佳 GradientBoostingRegressor
best_gbr = grid_search.best_estimator_

# 拟合整个训练集
start_time = time.time()
best_gbr.fit(X_train_scaled, y_train)
end_time = time.time()
gbr_training_time = end_time - start_time

# 交叉验证
gbr_cv_scores = cross_val_score(best_gbr, X_train_scaled, y_train, cv=5, scoring='r2')

# 测试集 R² 得分
gbr_test_score = best_gbr.score(X_test_scaled, y_test)

In [19]:
from IPython.display import Markdown

# 结果数据
results = {
    "模型": ["随机森林", "AdaBoost", "Gradient Boosting"],
    "训练时间 (秒)": [rf_training_time, ada_training_time, gbr_training_time],
    "平均交叉验证 R² 得分": [rf_cv_scores.mean(), ada_cv_scores.mean(), gbr_cv_scores.mean()],
    "测试集 R² 得分": [rf_test_score, ada_test_score, gbr_test_score],
}

# 创建 Markdown 格式的表格
table = "| 模型              | 训练时间 (秒) | 平均交叉验证 R² 得分 | 测试集 R² 得分 |\n"
table += "|------------------|---------------|----------------------|----------------|\n"
for i in range(len(results["模型"])):
    table += f"| {results['模型'][i]:<16} | {results['训练时间 (秒)'][i]:<13.2f} | {results['平均交叉验证 R² 得分'][i]:<20.4f} | {results['测试集 R² 得分'][i]:<14.4f} |\n"

# 显示 Markdown 表格
Markdown(table)

| 模型              | 训练时间 (秒) | 平均交叉验证 R² 得分 | 测试集 R² 得分 |
|------------------|---------------|----------------------|----------------|
| 随机森林             | 1.57          | 0.7822               | 0.7749         |
| AdaBoost         | 5.04          | 0.5542               | 0.5415         |
| Gradient Boosting | 12.67         | 0.8352               | 0.8383         |


### 7. 结论

在本分析中，我们成功地构建并优化了 **GradientBoostingRegressor** 模型来预测加利福尼亚房价。通过使用 **GridSearchCV** 进行超参数调优并实现 **提前停止**，我们提高了模型的表现。与其他集成方法（如 **随机森林** 和 **AdaBoost**）进行比较时，Gradient Boosting 展现出了更高的预测准确性，尽管训练时间较长。

**关键要点：**

- **Gradient Boosting** 是一种强大的集成技术，通常能提供较高的预测性能，但可能需要精心的超参数调优和更多的计算资源。
- **随机森林** 提供了一种稳健的替代方案，性能稍逊色，但训练时间更短。
- **AdaBoost** 适用于计算效率优先的场景，尽管可能牺牲一些准确性。

未来的工作可以考虑探索其他超参数优化技术，如 **RandomizedSearchCV** 或 **贝叶斯优化**，以提高调优效率。此外，尝试不同的特征工程方法或引入更先进的模型（如 **XGBoost** 或 **LightGBM**）可能进一步提高性能。